Hi all. This Note book checks data, and LightBGM template

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:
#Data Read
data_test=pd.read_csv("../input/g-research-crypto-forecasting/example_test.csv")
print("test_data")
data_test_=data_test.sort_values("Asset_ID")
data_test_.head()
data_test_.describe().T.style.background_gradient(cmap = 'Blues')\
                          .bar(subset = ["mean",], color = 'lightgreen')\
                           .bar(subset = ["std"], color = '#ee1f5f')

In [ ]:
data_train=pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
print("train_data")
data_train_=data_train.sort_values("Asset_ID")
data_train_.head()
data_train_.describe().T.style.background_gradient(cmap = 'Blues')\
                          .bar(subset = ["mean",], color = 'lightgreen')\
                           .bar(subset = ["std"], color = '#ee1f5f')

In [ ]:
#Check Null
def fillCheck(df):
        n_val=df.isnull().sum()
        pre=100*df.isnull().sum()/len(df)
        tdble=pd.concat([n_val,pre],axis=1)
        ren_colum=tdble.rename(columns={0:"Null Persent",1:"%"})
        return ren_colum

In [ ]:
print("train_data")
fillCheck(data_train_) 

In [ ]:
print("test_data")
fillCheck(data_test_)

In [ ]:
#Data_Split
df_train,df_val=train_test_split(data_train_,test_size=0.3)

X_train=df_train.drop("Target",axis=1)
y_train=df_train["Target"]

X_val=df_val.drop("Target",axis=1)
y_val=df_val["Target"]

In [ ]:
#LGB Modeling
params = {
    'random_state':100,
    'verbose':0,
    #Learning model indicators
    'metrics' : "auc"
}


train_d=lgb.Dataset(X_train,y_train)
val_d=lgb.Dataset(X_val,y_val)

model=lgb.train(params,train_d,valid_sets=val_d,num_boost_round=100,early_stopping_rounds=100)

In [ ]:
#fitting
data_x=data_test_.drop("group_num",axis=1)
data_X=data_x.drop("row_id",axis=1)

pred=model.predict(data_X)

In [ ]:
#submit
PassengerId=data_test_["Asset_ID"]
print(PassengerId.shape)
print(PassengerId.head())
Predictions=pd.DataFrame(data=pred,index=PassengerId,columns=['Target'])
Predictions.to_csv("submission.csv")